In [10]:
import pandas as pd
import math

# read data
popFracDF = pd.read_csv('iowa_county_popfrac.csv')
stateBenefits = pd.read_csv('iowa_UI_claims.csv')
countyClaims = pd.read_csv('iowa_county_initial_claims.csv')

# make dicts
popFrac = dict([ (row.County, row.Fraction) for row in popFracDF.itertuples() ])
totClaims = dict([ (row['Month'], row['Initial Claims']) for idx,row in stateBenefits.iterrows()])

# compute the Z-scores
zscores = pd.DataFrame(index = countyClaims['County FIP'])
date = '10/31/2013'
for idx, row in countyClaims.iterrows():
    month = row['Month']
    if month != date: continue
    name = row['County']
    FIP = row['County FIP']
    expected = popFrac[name]*totClaims[month]
    name += ' County, IA'
    stddev = math.sqrt(expected)
    zscores.at[FIP,'zscore'] = (row['Initial Claims'] - expected)/stddev
    zscores.at[FIP,'name'] = name
    
import folium
m = folium.Map(location=[42, -93.6], zoom_start=7)
m.choropleth(geo_data='../iowa-counties.geojson',
            name='choropleth',
            data=zscores,
            key_on='feature.properties.name',
            columns=['name', 'zscore'],
            fill_color='BuPu',
            fill_opacity=0.7)
m

In [6]:
m